In [326]:
import pandas as pd

## 데이터 세트 병합 전처리
### 수면장애 데이터 불러오기

In [327]:
# 데이터 불러오기
data = pd.read_csv('data1/수면장애 관련 주상병별 진료정보_2020.csv')
data

,주상병명,진료년도,시도,시군구,진료인원(명),시군구2
0,비기질성 수면장애,2021,서울특별시,종로구,1202,종로구
1,비기질성 수면장애,2021,서울특별시,중구,1280,중구
2,비기질성 수면장애,2021,서울특별시,용산구,1955,용산구
3,비기질성 수면장애,2021,서울특별시,성동구,2256,성동구
4,비기질성 수면장애,2021,서울특별시,광진구,2351,광진구
...,...,...,...,...,...,...
78,비기질성 수면장애,2021,경기도,포천시,1250,포천시
79,비기질성 수면장애,2021,경기도,여주시,969,여주시
80,비기질성 수면장애,2021,경기도,연천군,379,연천군
81,비기질성 수면장애,2021,경기도,가평군,770,가평군


In [328]:
# 진료년도 컬럼 삭제
data.drop(columns='진료년도', inplace=True)
data

,주상병명,시도,시군구,진료인원(명),시군구2
0,비기질성 수면장애,서울특별시,종로구,1202,종로구
1,비기질성 수면장애,서울특별시,중구,1280,중구
2,비기질성 수면장애,서울특별시,용산구,1955,용산구
3,비기질성 수면장애,서울특별시,성동구,2256,성동구
4,비기질성 수면장애,서울특별시,광진구,2351,광진구
...,...,...,...,...,...
78,비기질성 수면장애,경기도,포천시,1250,포천시
79,비기질성 수면장애,경기도,여주시,969,여주시
80,비기질성 수면장애,경기도,연천군,379,연천군
81,비기질성 수면장애,경기도,가평군,770,가평군


In [329]:
# 중구 중복 처리
data.loc[(data['시도']=='서울특별시')&(data['시군구']=='중구'), '시군구'] = '서울중구'
data.loc[(data['시도']=='인천광역시')&(data['시군구']=='중구'), '시군구'] = '인천중구'

# 시군구 대분류(시군구2) 중구 중복 처리
data.loc[(data['시도']=='서울특별시')&(data['시군구2']=='중구'), '시군구2'] = '서울중구'
data.loc[(data['시도']=='인천광역시')&(data['시군구2']=='중구'), '시군구2'] = '인천중구'

In [330]:
# 시군구 고유값인지 확인
if (data['시군구'].value_counts() > 1).any():
    print('시군구 고유값이 아닙니다.')
else:
    print('시군구 고유값입니다.')

시군구 고유값입니다.


In [331]:
# 중복된 시군구명 행을 첫 행만 남기고 나머지 삭제
data = data.drop_duplicates(subset=['시군구2'], keep='first')

# 빈도 컬럼을 중복된 시군구명의 평균값으로 대체
data['진료인원(명)'] = data.groupby('시군구2')['진료인원(명)'].transform('mean')

# 중복된 시군구명 행을 다시 삭제하여 평균값이 반영된 첫 행만 남기기
data = data.drop_duplicates(subset=['시군구2'], keep='first')

C:\Users\user\AppData\Local\Temp\ipykernel_5600\473994374.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['진료인원(명)'] = data.groupby('시군구2')['진료인원(명)'].transform('mean')


In [332]:
if (data['시군구2'].value_counts() > 1).any():
    print('중복된 시군구2 값:', data['시군구2'].value_counts()[data['시군구2'].value_counts() > 1].index.tolist())
else:
    print('중복없음')

중복없음


### 야식(배달량) 데이터세트 전처리

In [333]:
# 주거 특성별 배달 데이터 불러오기
data2 = pd.read_csv('data1/delivery.csv', index_col=0)
data2

,년,광역시도명,시군구명,세대구성대분류명,빈도,시군구2
0,2020,서울특별시,종로구,1세대 가구,8752.0,종로구
1,2020,서울특별시,종로구,1세대 가구,3162.0,종로구
2,2020,서울특별시,종로구,1세대 가구,2628.0,종로구
3,2020,서울특별시,종로구,1세대 가구,836.0,종로구
4,2020,서울특별시,종로구,1세대 가구,1406.0,종로구
...,...,...,...,...,...,...
59087,2020,경기도,양평군,비친족 가구,22.0,양평군
59088,2020,경기도,양평군,비친족 가구,10.0,양평군
59089,2020,경기도,양평군,비친족 가구,5.0,양평군
59090,2020,경기도,양평군,비친족 가구,5.0,양평군


In [334]:
# 년도 컬럼 삭제
data2.drop(columns=['년'], inplace=True)

# 광역시도명 -> 시도/ 시군구명 -> 시군구로 컬럼명 변경
data2.rename(columns={'광역시도명':'시도'}, inplace=True)
data2.rename(columns={'시군구명':'시군구'}, inplace=True)

In [335]:
# 중구 중복 처리
data2.loc[(data2['시도']=='서울특별시')&(data2['시군구']=='중구'), '시군구'] = '서울중구'
data2.loc[(data2['시도']=='인천광역시')&(data2['시군구']=='중구'), '시군구'] = '인천중구'

# 중구 중복 처리
data2.loc[(data2['시도']=='서울특별시')&(data2['시군구2']=='중구'), '시군구2'] = '서울중구'
data2.loc[(data2['시도']=='인천광역시')&(data2['시군구2']=='중구'), '시군구2'] = '인천중구'

In [336]:
# 세대구성대분류명 확인
data2['세대구성대분류명'].unique()

array(['1세대 가구', '2세대 가구', '3세대 가구', '4세대 이상 가구', '1인 가구', '비친족 가구'],
      dtype=object)

In [337]:
# 시군구별 세대구성대분류명의 빈도 평균 계산 (컬럼화)
data_pivot = data2.pivot_table(index='시군구', columns='세대구성대분류명', values='빈도', aggfunc='mean')

# 인덱스 초기화
data_pivot = data_pivot.reset_index()

# 세대구성대분류 대체
data_final = data2.drop(columns=['세대구성대분류명', '빈도']).drop_duplicates(subset=['시군구'])

# 기존 데이터와 병합
data2 = pd.merge(data_final, data_pivot, on='시군구', how='left')
data2

,시도,시군구,시군구2,1세대 가구,1인 가구,2세대 가구,3세대 가구,4세대 이상 가구,비친족 가구
0,서울특별시,종로구,종로구,348.633333,5198.666667,507.567407,119.103333,6.186667,380.753333
1,서울특별시,서울중구,서울중구,314.186667,4565.666667,427.126667,94.586667,5.173333,381.146667
2,서울특별시,용산구,용산구,553.745556,7378.266667,792.157778,158.685000,6.453333,709.946667
3,서울특별시,성동구,성동구,712.411111,8519.066667,1135.387407,228.603333,7.206667,541.346667
4,서울특별시,광진구,광진구,772.342222,12462.266667,1296.900000,247.738333,6.533333,688.546667
...,...,...,...,...,...,...,...,...,...
72,경기도,포천시,포천시,426.711111,3819.666667,523.937778,127.560000,8.086667,329.746667
73,경기도,여주시,여주시,332.723333,2659.666667,392.294074,116.565000,10.806667,223.746667
74,경기도,연천군,연천군,138.875556,1242.066667,140.801481,47.023333,5.226667,115.013333
75,경기도,가평군,가평군,205.344444,1801.266667,195.772593,57.165000,5.833333,144.146667


In [338]:
# 중복된 시군구명 행을 첫 행만 남기고 나머지 삭제
data2 = data2.drop_duplicates(subset=['시군구2'], keep='first')

# 빈도 컬럼을 중복된 시군구명의 평균값으로 대체
columns_to_transform = ['1세대 가구', '2세대 가구', '3세대 가구', '4세대 이상 가구', '1인 가구', '비친족 가구']
data2[columns_to_transform] = data2.groupby('시군구2')[columns_to_transform].transform('mean')

# 중복된 시군구명 행을 다시 삭제하여 평균값이 반영된 첫 행만 남기기
data2 = data2.drop_duplicates(subset=['시군구2'], keep='first')

C:\Users\user\AppData\Local\Temp\ipykernel_5600\331310453.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2[columns_to_transform] = data2.groupby('시군구2')[columns_to_transform].transform('mean')


In [339]:
# 시군구 고유값인지 확인
if (data2['시군구2'].value_counts() > 1).any():
    print('시군구 고유값이 아닙니다.')
else:
    print('시군구 고유값입니다.')

시군구 고유값입니다.


### 야식(시간대별 배달 주문량) 데이터 전처리

In [340]:
# 주문 가격 데이터 불러오기
data3 = pd.read_csv('data1/order_price.csv', index_col=0)
data3

,날짜,시간대별 시간,목적지 광역시도명,목적지 시군구명,평균주문금액,시군구2
0,2021,0,경기도,동두천시,72667,동두천시
1,2021,0,경기도,부천시,34877,부천시
2,2021,0,경기도,안산시 상록구,24420,안산시
3,2021,0,경기도,의정부시,31600,의정부시
6,2021,0,서울특별시,구로구,34345,구로구
...,...,...,...,...,...,...
1026031,2021,23,경기도,화성시,48630,화성시
1026037,2021,23,서울특별시,관악구,17100,관악구
1026038,2021,23,서울특별시,금천구,21960,금천구
1026039,2021,23,서울특별시,도봉구,5850,도봉구


In [341]:
# 날짜, 시군구2 컬럼 삭제
data3.drop(columns=['날짜'], inplace=True)

# 목적지 광역시도명 -> 시도, 목적지 시군구명 -> 시군구로 컬럼명 변경
data3.rename(columns={'목적지 광역시도명':'시도'}, inplace=True)
data3.rename(columns={'목적지 시군구명':'시군구'}, inplace=True)

In [342]:
# 중구 중복 처리
data3.loc[(data3['시도']=='서울특별시')&(data3['시군구']=='중구'), '시군구'] = '서울중구'
data3.loc[(data3['시도']=='인천광역시')&(data3['시군구']=='중구'), '시군구'] = '인천중구'

# 중구 중복 처리
data3.loc[(data3['시도']=='서울특별시')&(data3['시군구2']=='중구'), '시군구2'] = '서울중구'
data3.loc[(data3['시도']=='인천광역시')&(data3['시군구2']=='중구'), '시군구2'] = '인천중구'

In [343]:
# 야간평균주문금액

# 시간대별 시간 컬럼 삭제
data3.drop(columns=['시간대별 시간'], inplace=True)

# 시군구별 평균주문금액 계산
df_g = data3.groupby('시군구', as_index=False)['평균주문금액'].mean()

# 기존 데이터에서 '평균주문금액' 컬럼 대체
data3 = pd.merge(
    data3.drop(columns=['평균주문금액']).drop_duplicates(subset=['시군구']),
    df_g, on='시군구', how='left'
    )
data3

,시도,시군구,시군구2,평균주문금액
0,경기도,동두천시,동두천시,26882.143657
1,경기도,부천시,부천시,28329.552441
2,경기도,안산시 상록구,안산시,27154.460281
3,경기도,의정부시,의정부시,28458.182341
4,서울특별시,구로구,구로구,28139.356929
...,...,...,...,...
62,인천광역시,연수구,연수구,47500.000000
63,서울특별시,강동구,강동구,24000.000000
64,경기도,성남시 수정구,성남시,17500.000000
65,서울특별시,성동구,성동구,19055.000000


In [344]:
# 중복된 시군구명 행을 첫 행만 남기고 나머지 삭제
data3 = data3.drop_duplicates(subset=['시군구2'], keep='first')

# 빈도 컬럼을 중복된 시군구명의 평균값으로 대체
data3['평균주문금액'] = data3.groupby('시군구2')['평균주문금액'].transform('mean')

# 중복된 시군구명 행을 다시 삭제하여 평균값이 반영된 첫 행만 남기기
data3 = data3.drop_duplicates(subset=['시군구2'], keep='first')

C:\Users\user\AppData\Local\Temp\ipykernel_5600\4066627610.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['평균주문금액'] = data3.groupby('시군구2')['평균주문금액'].transform('mean')


In [345]:
# 시군구 고유값인지 확인
if (data3['시군구2'].value_counts() > 1).any():
    print('시군구 고유값이 아닙니다.')
else:
    print('시군구 고유값입니다.')

시군구 고유값입니다.


### 유흥업소 데이터 전처리

#### 서울

In [346]:
# 데이터 불러오기
alcohol_seoul = pd.read_csv('data1/서울_유흥업소_구별분류.csv')
alcohol_seoul

,상세영업상태코드,지번주소,사업장명,데이터갱신일자,구
0,1,서울특별시 강남구 역삼동 678-24 늘봄빌딩,골드코스트,2023-12-07 00:07:00.0,강남구
1,1,서울특별시 송파구 가락동 98-7,골드1,2023-12-03 22:06:00.0,송파구
2,2,서울특별시 마포구 합정동 382-22번지 지하1층,??,2018-08-31 23:59:59.0,마포구
3,1,서울특별시 영등포구 신길동 449-22 신성빌딩,린스,2022-10-31 22:02:00.0,영등포구
4,1,서울특별시 서초구 잠원동 12-21,만남,2022-12-05 22:09:00.0,서초구
...,...,...,...,...,...
4958,2,서울특별시 강동구 길동 415-7 지층,카카오톡,2021-12-04 23:05:00.0,강동구
4959,1,서울특별시 금천구 시흥동 124-1 지하1층,원투,2022-12-04 23:00:00.0,금천구
4960,1,서울특별시 영등포구 영등포동3가 17-9 스카이모텔,샤넬,2023-12-07 21:01:00.0,영등포구
4961,1,서울특별시 금천구 독산동 293-5,국빈관관광나이트크럽,2024-11-30 23:07:00.0,금천구


In [347]:
# 중구 -> 서울중구 변경
def change(name):
    if name == '중구':
        return '서울중구'
    return name

alcohol_seoul['구'] = alcohol_seoul['구'].apply(change)

In [348]:
# 결측치 제거
alcohol_seoul = alcohol_seoul.dropna(subset='구')

In [349]:
# 사업장 수 카운트
alcohol_seoul2 = alcohol_seoul.groupby(by='구', as_index=False).count()

In [350]:
# 사업장 수 컬럼만 남기기
alcohol_seoul2 = alcohol_seoul2.drop(['상세영업상태코드','지번주소','데이터갱신일자'], axis=1)
alcohol_seoul2.rename(columns={'사업장명':'유흥업소 개수'}, inplace=True)
alcohol_seoul2

,구,유흥업소 개수
0,강남구,658
1,강동구,288
2,강북구,154
3,강서구,232
4,관악구,423
5,광진구,26
6,구로구,76
7,금천구,102
8,노원구,44
9,도봉구,32


#### 인천&경기

In [351]:
# 데이터 불러오기
alcohol_gg = pd.read_csv('data1/인천경기_유흥업소_찐찐.csv')

alcohol_gg

,상세영업상태명,도로명전체주소,사업장명,구,시도,시군구2
0,영업,인천광역시 옹진군 백령면 백령로 298-1,장미노래클럽,NaN,인천광역시,옹진군
1,영업,인천광역시 옹진군 영흥면 영흥남로 307,빠샤 노래주점,NaN,인천광역시,옹진군
2,영업,인천광역시 옹진군 영흥면 영흥북로35번길 21,뉴 차차차 노래클럽,NaN,인천광역시,옹진군
3,영업,인천광역시 옹진군 백령면 백령로 282-1,퍼펙트,NaN,인천광역시,옹진군
4,영업,"인천광역시 옹진군 백령면 백령로 287, 1층",텔레파시,NaN,인천광역시,옹진군
...,...,...,...,...,...,...
6285,영업,"경기도 파주시 와석순환로515번길 69-1, 202호 (와동동)",라스베가스 노래클럽,NaN,경기도,파주시
6286,영업,"경기도 파주시 와석순환로515번길 69-1, 203호 (와동동)",라스베가스 노래타운,NaN,경기도,파주시
6287,영업,"경기도 평택시 고덕중앙로 200-17, 베네치아 지하1층 B101~B103호 (고덕동)",준코뮤직타운 평택고덕점,NaN,경기도,평택시
6288,영업,"인천광역시 연수구 능허대로267번길 29, 라마다송도호텔 지하1층 일부호 (동춘동)",에스파스,연수구,인천광역시,연수구


In [352]:
# 중구 -> 인천중구 변경
def change2(name):
    if name == '중구':
        return '인천중구'
    return name

alcohol_gg['구'] = alcohol_gg['시군구2'].apply(change2)

In [353]:
# 사업장 수 카운트
alcohol_gg2 = alcohol_gg.groupby(by='구', as_index=False).count()

In [354]:
# 사업장 수 컬럼만 남기기
alcohol_gg2 = alcohol_gg2.drop(['상세영업상태명','도로명전체주소','시도','시군구2'], axis=1)
alcohol_gg2.rename(columns={'사업장명':'유흥업소 개수'}, inplace=True)
alcohol_gg2

,구,유흥업소 개수
0,가평군,60
1,강화군,19
2,계양구,91
3,고양시,43
4,광명시,139
5,광주시,35
6,구리시,179
7,군포시,96
8,김포시,148
9,남양주시,107


In [355]:
alcohol_gg2[alcohol_gg2['구'] =='안양시']

,구,유흥업소 개수
21,안양시,338


In [356]:
# 유흥업소 데이터 통합
alcohol = pd.concat([alcohol_seoul2,alcohol_gg2], ignore_index=True)
alcohol.rename(columns={'구':'시군구2'}, inplace=True)
alcohol

,시군구2,유흥업소 개수
0,강남구,658
1,강동구,288
2,강북구,154
3,강서구,232
4,관악구,423
...,...,...
59,파주시,156
60,평택시,633
61,포천시,141
62,하남시,55


### 활동량 데이터 전처리

In [357]:
# 데이터 불러오기
data4 = pd.read_csv('data1/activity.csv', index_col=0)
data4

,기준연월,시도명,시군구명,시군구별시설수,시군구별인구수,인당시설수,시군구2
0,202008,서울특별시,종로구,81,149962,0.00054,종로구
1,202008,서울특별시,중구,26,125990,0.00021,중구
2,202008,서울특별시,용산구,28,229362,0.00012,용산구
3,202008,서울특별시,성동구,28,296183,0.00009,성동구
4,202008,서울특별시,광진구,25,348647,0.00007,광진구
...,...,...,...,...,...,...,...
1017,202012,경기도,포천시,102,147274,0.00069,포천시
1018,202012,경기도,여주시,69,111897,0.00062,여주시
1019,202012,경기도,연천군,78,43516,0.00179,연천군
1020,202012,경기도,가평군,76,62377,0.00122,가평군


In [358]:
# 9월 데이터만 추출
act09 = data4[data4['기준연월'] == 202009]
act09

,기준연월,시도명,시군구명,시군구별시설수,시군구별인구수,인당시설수,시군구2
229,202009,서울특별시,종로구,81,149952,0.00054,종로구
230,202009,서울특별시,중구,26,125800,0.00021,중구
231,202009,서울특별시,용산구,28,229786,0.00012,용산구
232,202009,서울특별시,성동구,28,295591,0.00009,성동구
233,202009,서울특별시,광진구,25,348064,0.00007,광진구
...,...,...,...,...,...,...,...
330,202009,경기도,포천시,102,147803,0.00069,포천시
331,202009,경기도,여주시,69,111512,0.00062,여주시
332,202009,경기도,연천군,78,43590,0.00179,연천군
333,202009,경기도,가평군,76,62551,0.00122,가평군


In [359]:
# 시군구, 시도 컬럼명 변경
act09.rename(columns={'시군구명':'시군구'}, inplace=True)
act09.rename(columns={'시도명':'시도'}, inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_5600\3240045862.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  act09.rename(columns={'시군구명':'시군구'}, inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_5600\3240045862.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  act09.rename(columns={'시도명':'시도'}, inplace=True)


In [360]:

# 중복된 시군구명 행을 첫 행만 남기고 나머지 삭제
act09 = act09.drop_duplicates(subset=['시군구'], keep='first')

# 빈도 컬럼을 중복된 시군구명의 총합 대체(밑에 주석도 총합인데 평균으로 되어있어요)
# 인당 시설수 계산 다시해야할 것 같긴한데.. 일단 이대로 진행합니다.
list = ['시군구별시설수','시군구별인구수','인당시설수']
act09[list] = act09.groupby('시군구')[list].transform('sum')

# 중복된 시군구명 행을 다시 삭제하여 총합이 반영된 첫 행만 남기기
act09 = act09.drop_duplicates(subset=['시군구'], keep='first')

C:\Users\user\AppData\Local\Temp\ipykernel_5600\2621990506.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  act09[list] = act09.groupby('시군구')[list].transform('sum')


In [361]:
# 중구 중복 처리
act09.loc[(act09['시도']=='서울특별시')&(act09['시군구']=='중구'), '시군구'] = '서울중구'
act09.loc[(act09['시도']=='인천광역시')&(act09['시군구']=='중구'), '시군구'] = '인천중구'

In [362]:
# 시군구 고유값인지 확인
if (act09['시군구2'].value_counts() > 1).any():
    print('시군구 고유값이 아닙니다.')
else:
    print('시군구 고유값입니다.')

시군구 고유값입니다.


In [363]:
act09.drop(columns=['시군구'], inplace=True)

### 범죄율 데이터 전처리

In [364]:
data5 = pd.read_csv('data1/범죄 빈발지 데이터 2020.csv',encoding='cp949')
data5

,신고년도,시도,시군구,총신고수,가정폭력신고수,데이트폭력신고수,성폭력신고수,몰래카메라신고수,살인신고수,강도신고수,절도신고수,폭력신고수
0,2020,경기도,가평군,1141,184,59,28,0,0,2,273,595
1,2020,경기도,고양시,26557,6148,845,648,9,12,10,5746,13175
2,2020,경기도,과천시,603,165,7,24,0,0,0,183,224
3,2020,경기도,광명시,26100,6314,1080,666,0,2,20,6100,11918
4,2020,경기도,광주시,6238,2260,253,111,1,0,0,1283,2331
...,...,...,...,...,...,...,...,...,...,...,...,...
61,2020,인천광역시,부평구,45808,8272,3740,1257,3,14,45,8559,23921
62,2020,인천광역시,서구,16628,4931,992,370,0,4,3,2778,7550
63,2020,인천광역시,연수구,13805,3851,910,388,1,3,0,2974,5679
64,2020,인천광역시,옹진군,202,35,9,6,0,1,0,27,124


In [365]:
# 중구 중복 처리
data5.loc[(data5['시도']=='서울특별시')&(data5['시군구']=='중구'), '시군구'] = '서울중구'
data5.loc[(data5['시도']=='인천광역시')&(data5['시군구']=='중구'), '시군구'] = '인천중구'

In [366]:
# 총신고수 제외 컬럼 삭제
data5 = data5[['시군구', '총신고수']]
data5.rename(columns={'시군구': '시군구2'}, inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_5600\1838929866.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data5.rename(columns={'시군구': '시군구2'}, inplace=True)


### 교통량 데이터 전처리

In [367]:
# 데이터 불러오기
data6 = pd.read_csv('data1/야간교통량 평균.csv')
data6

,도로명,구 이름,유입야간교통량평균
0,A-01 성산로(금화터널,서대문구,746.545455
1,A-02 사직로(사직터널,종로구,867.636364
2,A-03 자하문로(자하문터널,종로구,347.454545
3,A-04 대사관로(삼청터널,성북구,80.545455
4,A-05 율곡로(안국역,종로구,695.909091
...,...,...,...
121,D-41 노들로,동작구,646.909091
122,D-42 테헤란로(선릉역,강남구,813.000000
123,D-43 강남대로(신사역,강남구,1195.181818
124,D-44 백제고분로(종합운동장,송파구,531.454545


In [368]:
data6.loc[data6['구 이름']=='중구','구 이름'] = '서울중구'

In [369]:
data6 = data6.groupby(by='구 이름',as_index=False)['유입야간교통량평균'].mean()

data6

,구 이름,유입야간교통량평균
0,강남구,1000.161616
1,강동구,1206.636363
2,강서구,944.324675
3,관악구,801.909091
4,광진구,868.400000
5,구로구,550.068182
6,금천구,590.045454
7,노원구,856.781818
8,도봉구,666.242424
9,동작구,900.750000


In [370]:
# 구이름 -> 시군구 컬럼명 변경
data6.rename(columns={'구 이름':'시군구2'}, inplace=True)

### 공시지가 데이터 전처리

In [371]:
data7 = pd.read_csv('data1/공시지가_레알찐.csv', encoding='cp949')
data7

,시군구명,공시지가,공시지가2
0,가평군,1.425912e+05,0.008486
1,강남구,1.680318e+07,1.000000
2,강동구,5.667941e+06,0.337314
3,강북구,3.519811e+06,0.209473
4,강서구,4.261157e+06,0.253592
...,...,...,...
61,평택시,6.349830e+05,0.037789
62,포천시,1.834384e+05,0.010917
63,하남시,2.540962e+06,0.151219
64,화성시,4.114059e+05,0.024484


In [372]:
# 시군구명 -> 시군구로 컬럼명 변경
data7.rename(columns={'시군구명':'시군구2'}, inplace=True)

## 데이터 병합

In [373]:
data3.drop(columns=['시도'], inplace=True)
data2.drop(columns=['시도'], inplace=True)
act09.drop(columns=['시도'], inplace=True)

In [374]:
# 데이터 병합
data_merge = pd.merge(data, data2, on='시군구2', how='left').merge(data3, on='시군구2', how='left')
data_merge = pd.merge(data_merge, alcohol, on='시군구2', how='left')
data_merge = pd.merge(data_merge, act09, on='시군구2', how='left')
data_merge = pd.merge(data_merge, data5, on='시군구2', how='left')
data_merge = pd.merge(data_merge, data6, on='시군구2', how='left')
data_merge = pd.merge(data_merge, data7, on='시군구2', how='left')

# 병합된 데이터 확인
data_merge.head()

,주상병명,시도,시군구_x,진료인원(명),시군구2,시군구_y,1세대 가구,1인 가구,2세대 가구,3세대 가구,...,평균주문금액,유흥업소 개수,기준연월,시군구별시설수,시군구별인구수,인당시설수,총신고수,유입야간교통량평균,공시지가,공시지가2
0,비기질성 수면장애,서울특별시,종로구,1202.0,종로구,종로구,348.633333,5198.666667,507.567407,119.103333,...,24024.285714,439.0,202009.0,81.0,149952.0,0.00054,6834.0,524.102273,9.045009e+06,0.538291
1,비기질성 수면장애,서울특별시,서울중구,1280.0,서울중구,서울중구,314.186667,4565.666667,427.126667,94.586667,...,27154.801724,656.0,NaN,NaN,NaN,NaN,10584.0,441.578512,1.399032e+07,0.832600
2,비기질성 수면장애,서울특별시,용산구,1955.0,용산구,용산구,553.745556,7378.266667,792.157778,158.685000,...,26419.737778,88.0,202009.0,28.0,229786.0,0.00012,6923.0,901.952273,9.666904e+06,0.575302
3,비기질성 수면장애,서울특별시,성동구,2256.0,성동구,성동구,712.411111,8519.066667,1135.387407,228.603333,...,19055.000000,33.0,202009.0,28.0,295591.0,0.00009,6164.0,838.563636,6.484237e+06,0.385893
4,비기질성 수면장애,서울특별시,광진구,2351.0,광진구,광진구,772.342222,12462.266667,1296.900000,247.738333,...,NaN,26.0,202009.0,25.0,348064.0,0.00007,18288.0,868.400000,5.778178e+06,0.343874


In [375]:
data_merge.head()

,주상병명,시도,시군구_x,진료인원(명),시군구2,시군구_y,1세대 가구,1인 가구,2세대 가구,3세대 가구,...,평균주문금액,유흥업소 개수,기준연월,시군구별시설수,시군구별인구수,인당시설수,총신고수,유입야간교통량평균,공시지가,공시지가2
0,비기질성 수면장애,서울특별시,종로구,1202.0,종로구,종로구,348.633333,5198.666667,507.567407,119.103333,...,24024.285714,439.0,202009.0,81.0,149952.0,0.00054,6834.0,524.102273,9.045009e+06,0.538291
1,비기질성 수면장애,서울특별시,서울중구,1280.0,서울중구,서울중구,314.186667,4565.666667,427.126667,94.586667,...,27154.801724,656.0,NaN,NaN,NaN,NaN,10584.0,441.578512,1.399032e+07,0.832600
2,비기질성 수면장애,서울특별시,용산구,1955.0,용산구,용산구,553.745556,7378.266667,792.157778,158.685000,...,26419.737778,88.0,202009.0,28.0,229786.0,0.00012,6923.0,901.952273,9.666904e+06,0.575302
3,비기질성 수면장애,서울특별시,성동구,2256.0,성동구,성동구,712.411111,8519.066667,1135.387407,228.603333,...,19055.000000,33.0,202009.0,28.0,295591.0,0.00009,6164.0,838.563636,6.484237e+06,0.385893
4,비기질성 수면장애,서울특별시,광진구,2351.0,광진구,광진구,772.342222,12462.266667,1296.900000,247.738333,...,NaN,26.0,202009.0,25.0,348064.0,0.00007,18288.0,868.400000,5.778178e+06,0.343874


In [379]:
data_merge.drop(columns=['시군구_x','시군구_y','시군구','기준연월'], inplace=True)

In [380]:
# 데이터 저장
data_merge.to_csv('data1/통합데이터.csv', index=False)